In [1]:
from glob import glob
from tqdm import tqdm
import pandas as pd
import tensorflow.compat.v1 as tf
tf.logging.set_verbosity(tf.logging.ERROR)  # Hide TF deprecation messages
import matplotlib.pyplot as plt
import trimesh
import random
import numpy as np
import pickle
import modules
import data_utils

2023-07-23 10:52:02.780247: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
import datetime
from tensorflow import summary as s
# log_dir = "logs/text_gen/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#summary_writer = s.FileWriter(log_dir)

In [3]:
BATCH_SIZE=8

In [4]:
chair_meshes_paths = list(glob("chairs_ngon/*.obj"))
# chair_meshes_paths = [["Chair", path] for path in chair_meshes_paths]
# tables_meshes_paths = list(glob("tables_ngon/*.obj"))
# tables_meshes_paths = [["Table", path] for path in tables_meshes_paths]
# chair_meshes_paths.extend(tables_meshes_paths)
# paths = chair_meshes_paths.copy()
# random.shuffle(paths)

In [5]:
with open("chairs_split_dict.pickle", 'rb') as f:
    chairs_split_dict = pickle.load(f)

In [6]:
# with open("tables_split_dict.pickle", 'rb') as f:
#     tables_split_dict = pickle.load(f)

In [7]:
chairs_train = []
chairs_val = []
chairs_test = []
for c in chair_meshes_paths: 
    try:
        split = chairs_split_dict[c.split("/")[-1].replace(".obj", "")]
    except KeyError:
#         print(c.split("/")[-1].replace(".obj", ""))
        continue
    if split =='train':
        chairs_train.append(c)
    elif split =='val':
        chairs_val.append(c)
    else:
        chairs_test.append(c)
# print(len(chairs_train))
# print(len(chairs_val))
# print(len(chairs_test))

In [8]:
# tables_train = []
# tables_val = []
# tables_test = []
# for c in tables_meshes_paths:
#     try:
#         split = tables_split_dict[c.split("/")[-1].replace(".obj", "")]
#     except KeyError:
# #         print(c.split("/")[-1].replace(".obj", ""))
#         continue
#     if split =='train':
#         tables_train.append(c)
#     elif split =='val':
#         tables_val.append(c)
#     else:
#         tables_test.append(c)
# # print(len(tables_train))
# # print(len(tables_val))
# # print(len(tables_test))

In [9]:
# chairs_train.extend(tables_train)
train_paths = chairs_train.copy()
random.shuffle(train_paths)
# train_paths = train_paths[:10]

In [10]:
TRAIN_SIZE = len(train_paths)

In [11]:
# chairs_val.extend(tables_val)
val_paths = chairs_val.copy()
random.shuffle(val_paths)

In [12]:
VAL_SIZE = len(val_paths)

In [13]:
captions = pd.read_csv("captions_tablechair.csv").dropna()

In [14]:
# captions

In [15]:
max_length = []
for c in captions['description'].values:
    max_length.append(len(c.split(" ")))
max_length = np.array(max_length)
(max_length < 30).sum()/len(captions['description'].values)

0.9483664640781337

In [16]:
max_length = 30

In [17]:
train_captions=[]
for index, row in captions.iterrows():
    try:
        # if row["category"]=="Table":
        #     if tables_split_dict[row["modelId"]]=='train':
        #         train_captions.append(row['description'])
        if row["category"]=="Chair":
            if chairs_split_dict[row["modelId"]]=='train':
                train_captions.append(row['description'])
    except KeyError:
        continue
# print(len(train_captions))

In [18]:
from tensorflow.keras.preprocessing.text import Tokenizer
tk = Tokenizer()
tk.fit_on_texts(train_captions)

In [19]:
def text2shape(paths, captions, tokenizer):
    for path in paths:
        # with open(path, 'rb') as obj_file:
        mesh_dict = data_utils.load_process_mesh(path)
#         mesh_dict['class_label'] = 18 if cls=="Chair" else 49
        if len(mesh_dict['vertices'])>500:
            continue
        if len(mesh_dict['faces'])>2600:
            continue
        # mesh_dict = random_scaling(mesh_dict)
        try:
            text = captions[captions["modelId"]==path.split("/")[-1].replace(".obj", "")].sample(n=1)["description"].values[0]
        except:
            continue
        text = text.lower().replace("the", '').replace("a", '').replace("of", '').replace("for", '').replace("and", '').replace("to", '').replace("in", '')
        text = " ".join(text.split(" ")[:max_length])
        text = tokenizer.texts_to_sequences([text])[0]
        mesh_dict['text_feature'] = np.pad(text, (0,max_length-len(text)))
        yield mesh_dict

In [20]:
Text2ShapeDataset = tf.data.Dataset.from_generator(
        lambda:text2shape(train_paths, captions, tk),
        output_types={
            'vertices': tf.int32, 'faces': tf.int32,
#             'class_label': tf.int32,
            'text_feature': tf.int32},
        output_shapes={
            'vertices': tf.TensorShape([None, 3]), 'faces': tf.TensorShape([None]),
#             'class_label': tf.TensorShape(()),
            'text_feature':tf.TensorShape(max_length)})

In [21]:
vertex_model_dataset = data_utils.make_vertex_model_dataset(Text2ShapeDataset, apply_random_shift=False)
vertex_model_dataset = vertex_model_dataset.repeat()
vertex_model_dataset = vertex_model_dataset.padded_batch(BATCH_SIZE, padded_shapes=vertex_model_dataset.output_shapes)
vertex_model_dataset = vertex_model_dataset.prefetch(1)
it = vertex_model_dataset.make_initializable_iterator()
vertex_model_batch = it.get_next()
iterator_init_op_train = it.initializer

In [22]:
with tf.Session() as sess:
    sess.run([iterator_init_op_train])
    vv = sess.run([vertex_model_batch])
vv

2023-07-23 10:52:26.325487: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-07-23 10:52:26.437790: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1669] Found device 0 with properties: 
name: NVIDIA GeForce RTX 3090 major: 8 minor: 6 memoryClockRate(GHz): 1.74
pciBusID: 0000:65:00.0
2023-07-23 10:52:26.437854: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2023-07-23 10:52:26.472236: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2023-07-23 10:52:26.476093: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2023-07-23 10:52:26.476779: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcurand.so.10
2023-07-23 10:52:26.478564: I tensorflow/stream_executor/platform/

[{'vertices': array([[[164, 167,  18],
          [173, 167,  18],
          [164, 174,  18],
          ...,
          [  0,   0,   0],
          [  0,   0,   0],
          [  0,   0,   0]],
  
         [[ 65,  65,  57],
          [ 70,  65,  57],
          [ 65,  67,  57],
          ...,
          [189, 130, 196],
          [181, 130, 197],
          [184, 130, 197]],
  
         [[ 78,  60,  42],
          [ 79,  60,  42],
          [194,  60,  42],
          ...,
          [  0,   0,   0],
          [  0,   0,   0],
          [  0,   0,   0]],
  
         ...,
  
         [[ 84,  84,  15],
          [ 93,  84,  15],
          [143,  84,  15],
          ...,
          [  0,   0,   0],
          [  0,   0,   0],
          [  0,   0,   0]],
  
         [[ 81,  63,  46],
          [ 91,  63,  46],
          [151,  63,  46],
          ...,
          [  0,   0,   0],
          [  0,   0,   0],
          [  0,   0,   0]],
  
         [[ 71,  47,  46],
          [ 80,  47,  46],
          [1

In [23]:
Text2ShapeDatasetVal = tf.data.Dataset.from_generator(
        lambda:text2shape(val_paths, captions, tk),
        output_types={
            'vertices': tf.int32, 'faces': tf.int32,
#             'class_label': tf.int32,
            'text_feature': tf.int32},
        output_shapes={
            'vertices': tf.TensorShape([None, 3]), 'faces': tf.TensorShape([None]),
#             'class_label': tf.TensorShape(()),
            'text_feature':tf.TensorShape(max_length)})
vertex_model_dataset_val = data_utils.make_vertex_model_dataset(Text2ShapeDatasetVal, apply_random_shift=False)
vertex_model_dataset_val = vertex_model_dataset_val.repeat()
vertex_model_dataset_val = vertex_model_dataset_val.padded_batch(BATCH_SIZE, padded_shapes=vertex_model_dataset_val.output_shapes)
vertex_model_dataset_val = vertex_model_dataset_val.prefetch(1)
itv = vertex_model_dataset_val.make_initializable_iterator()
vertex_model_batch_val = itv.get_next()
iterator_init_op_val = itv.initializer

In [24]:
o = (i for i in text2shape(train_paths, captions, tk))
TRAIN_SIZE = sum(1 for _ in o)
o = (i for i in text2shape(val_paths, captions, tk))
VAL_SIZE = sum(1 for _ in o)

In [25]:
TRAIN_SIZE

1805

In [26]:
VAL_SIZE

330

In [27]:
vertex_model = modules.TextToVertexModel(
    decoder_config=dict(
      hidden_size=128,
      fc_size=512,
      num_heads=8,
      layer_norm=True,
      num_layers=24,
      dropout_rate=0.4,
      re_zero=True,
      memory_efficient=True
      ),
    path_to_embeddings="glove.6B/glove.6B.100d.txt",
    embedding_dims = 100,
    quantization_bits=8,
    tokenizer=tk,
    max_num_input_verts=500,  # number of vertices in the input mesh, if this is lower than the number of vertices in the mesh, there will be errors during training
    use_discrete_embeddings=True
)

Found 400000 word vectors.
Converted 5033 words (1171 misses)


In [28]:
vertex_model_pred_dist = vertex_model(vertex_model_batch, is_training=True)
vertex_model_loss = -tf.reduce_sum(
    vertex_model_pred_dist.log_prob(vertex_model_batch['vertices_flat']) *
    vertex_model_batch['vertices_flat_mask'])
vertex_samples = vertex_model.sample(
    BATCH_SIZE, context=vertex_model_batch, max_sample_length=500, top_p=0.95,
    recenter_verts=False, only_return_complete=False)

print(vertex_model_batch)
print(vertex_model_pred_dist)


{'vertices': <tf.Tensor 'IteratorGetNext:2' shape=(?, ?, 3) dtype=int32>, 'faces': <tf.Tensor 'IteratorGetNext:0' shape=(?, ?) dtype=int32>, 'text_feature': <tf.Tensor 'IteratorGetNext:1' shape=(?, 30) dtype=int32>, 'vertices_flat': <tf.Tensor 'IteratorGetNext:3' shape=(?, ?) dtype=int32>, 'vertices_flat_mask': <tf.Tensor 'IteratorGetNext:4' shape=(?, ?) dtype=float32>}
tfp.distributions.Categorical("vertex_model_2/vertex_model/create_dist/Categorical/", batch_shape=[?, ?], event_shape=[], dtype=int32)


In [29]:
vertex_model_pred_dist_val = vertex_model(vertex_model_batch_val)
vertex_model_loss_val = -tf.reduce_sum(
    vertex_model_pred_dist_val.log_prob(vertex_model_batch_val['vertices_flat']) *
    vertex_model_batch_val['vertices_flat_mask'])
vertex_samples_val = vertex_model.sample(
    BATCH_SIZE, context=vertex_model_batch_val, max_sample_length=500, top_p=0.95,
    recenter_verts=False, only_return_complete=False)

In [30]:
# with open("only_text_vars.pickle", 'rb') as f:
#     only_text_vars = pickle.load(f)

# text_vars = []
# for var in vertex_model.variables:
#     if '/'.join((var._variable._name).split('/')[1:]) in only_text_vars:
#         text_vars.append(var)
# text_vars=tuple(text_vars)
# # print(text_vars)

In [31]:
#import pickle
#with open("pretrained_vars.pickle", 'rb') as f:
#    common_vars = pickle.load(f)

#pretrained_vars = []
#for var in vertex_model.variables:
##    if '/'.join((var._variable._name).split('/')[1:]) in common_vars:
#        pretrained_vars.append(var)
#pretrained_vars=tuple(pretrained_vars)
# print(pretrained_vars)

In [32]:
#vertex_model_saver = tf.train.Saver(var_list=pretrained_vars)

In [33]:
# # Create face model
# face_model = modules.FaceModel(
#       encoder_config=dict(
#       hidden_size=128,
#       fc_size=512,
#       num_heads=8,
#       layer_norm=True,
#       num_layers=10,
#       dropout_rate=0.2,
#       re_zero=True,
#       memory_efficient=True,
#       ),
#   decoder_config=dict(
#       hidden_size=128,
#       fc_size=512,
#       num_heads=8,
#       layer_norm=True,
#       num_layers=14,
#       dropout_rate=0.2,
#       re_zero=True,
#       memory_efficient=True,
#       ),
#     class_conditional=False,
#     max_seq_length=1000, # number of faces in the input mesh, if this is lower than the number of vertices in the mesh, there will be errors during training
#     quantization_bits=8,
#     decoder_cross_attention=True,
#     use_discrete_vertex_embeddings=True,
# )

face_module_config=dict(
  encoder_config=dict(
      hidden_size=512,
      fc_size=2048,
      num_heads=8,
      layer_norm=True,
      num_layers=10,
      dropout_rate=0.2,
      re_zero=True,
      memory_efficient=True,
      ),
  decoder_config=dict(
      hidden_size=512,
      fc_size=2048,
      num_heads=8,
      layer_norm=True,
      num_layers=14,
      dropout_rate=0.2,
      re_zero=True,
      memory_efficient=True,
      ),
  class_conditional=False,
  decoder_cross_attention=True,
  use_discrete_vertex_embeddings=True,
  max_seq_length=8000,
  )
face_model=modules.FaceModel(**face_module_config)

# face_model_dataset = data_utils.make_face_model_dataset(
#     Text2ShapeDataset, apply_random_shift=True)
# face_model_dataset = face_model_dataset.repeat()
# face_model_dataset = face_model_dataset.padded_batch(
#     BATCH_SIZE, padded_shapes=face_model_dataset.output_shapes)
# face_model_dataset = face_model_dataset.prefetch(1)
# itf = face_model_dataset.make_initializable_iterator()
# face_model_batch = itf.get_next()
# f_iterator_init_op_train = itf.initializer

# face_model_pred_dist = face_model(face_model_batch)
# face_model_loss = -tf.reduce_sum(
#     face_model_pred_dist.log_prob(face_model_batch['faces']) *
#     face_model_batch['faces_mask'])
face_samples = face_model.sample(
    context=vertex_samples, max_sample_length=2000, top_p=0.95,
    only_return_complete=False)

In [34]:
# face_model_dataset_val = data_utils.make_face_model_dataset(
#     Text2ShapeDatasetVal, apply_random_shift=False)
# face_model_dataset_val  = face_model_dataset_val.repeat()
# face_model_dataset_val  = face_model_dataset_val.padded_batch(
#     BATCH_SIZE, padded_shapes=face_model_dataset_val.output_shapes)
# face_model_dataset_val = face_model_dataset_val.prefetch(1)
# itfv = face_model_dataset_val.make_initializable_iterator()
# face_model_batch_val = itfv.get_next()
# f_iterator_init_op_val = itfv.initializer

# face_model_pred_dist_val = face_model(face_model_batch_val)
# face_model_loss_val = -tf.reduce_sum(
#     face_model_pred_dist_val.log_prob(face_model_batch_val['faces']) *
#     face_model_batch_val['faces_mask'])
face_samples_val = face_model.sample(
    context=vertex_samples_val, max_sample_length=2000, top_p=0.95,
    only_return_complete=False)

In [35]:
face_model_saver = tf.train.Saver(var_list=face_model.variables)

In [ ]:
from tqdm import trange
import os

last_saver_vertex = tf.train.Saver(var_list=vertex_model.variables) # will keep last 5 epochs
best_saver_vertex = tf.train.Saver(var_list=vertex_model.variables, max_to_keep=2)  # only keep 1 best checkpoint (best on eval)

# last_saver_faces = tf.train.Saver(var_list=face_model.variables) # will keep last 5 epochs
# best_saver_faces = tf.train.Saver(var_list=face_model.variables, max_to_keep=2)  # only keep 1 best checkpoint (best on eval)


%matplotlib inline 
learning_rate = 3e-4
training_steps = 500
check_step_metrics = 50
check_step_samples = 50
EPOCHS = 500
optimizer = tf.train.AdamOptimizer(learning_rate)
#vertex_model_optim_op = optimizer.minimize(vertex_model_loss, var_list=text_vars)
vertex_model_optim_op = optimizer.minimize(vertex_model_loss, var_list=vertex_model.variables)
best_v_loss = 4194.5093 #4608.057 #float('inf')
# face_model_optim_op = optimizer.minimize(face_model_loss, var_list=face_model.variables)
# best_f_loss = float('inf')
#gstep = 0
# Training loop
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    last_saver_vertex.restore(sess, "text_vertex/last-141")
    # best_saver_faces.restore(sess, 'text_f/best-4')
    # best_saver_faces.restore(sess, "text_f/best-9")
    face_model_saver.restore(sess, "face_model/model")

    for e in range(142, EPOCHS):
        print("Epoch {}/{}".format(e + 1, EPOCHS))
        num_steps = (TRAIN_SIZE + BATCH_SIZE - 1) // BATCH_SIZE
        sess.run(iterator_init_op_train)
        # sess.run(f_iterator_init_op_train)
        t = trange(num_steps)
        loss_values = []
        # loss_values_f=[]
        for i in t:
            sess.run(vertex_model_optim_op)
            # sess.run(face_model_optim_op)

            loss_val = sess.run(vertex_model_loss)
            loss_values.append(loss_val)
            
            # loss_val_f = sess.run(face_model_loss)
            # loss_values_f.append(loss_val_f)
 
            # Log the loss in the tqdm progress bar
            t.set_postfix(loss='{:05.3f}'.format(loss_val))
        mean_loss = np.array(loss_values).mean()
        # mean_loss_f = np.array(loss_values_f).mean()
        print("- Train loss vertex: " + str(mean_loss)) 
        # print("- Train loss faces: " + str(mean_loss_f))
        
        last_save_path = os.path.join("text_vertex", 'last')
        last_saver_vertex.save(sess, last_save_path, global_step=e + 1)
        # last_save_path = os.path.join("text_f", 'last')
        # last_saver_faces.save(sess, last_save_path, global_step=e + 1)
    
        num_steps = (VAL_SIZE + BATCH_SIZE - 1) // BATCH_SIZE
        sess.run(iterator_init_op_val)
        # sess.run(f_iterator_init_op_val)
        loss_values = []
        # loss_values_f = []
        t = trange(num_steps)
        for i in t:
            loss_val = sess.run(vertex_model_loss_val)
            loss_values.append(loss_val)
            # # loss_val_f = sess.run(face_model_loss_val)
            # loss_values_f.append(loss_val_f)
            t.set_postfix(loss='{:05.3f}'.format(loss_val))
        mean_loss = np.array(loss_values).mean()
        # mean_loss_f = np.array(loss_values_f).mean()
        print("- Eval loss vertex: " + str(mean_loss)) 
        # print("- Eval loss faces: " + str(mean_loss_f))
 
        if mean_loss<=best_v_loss:
            best_v_loss = mean_loss
            best_save_path = os.path.join("text_vertex", 'best')
            best_save_path = best_saver_vertex.save(sess, best_save_path, global_step=e + 1)
            print("- Found new best vertex model, saving in {}".format(best_save_path))
        # if mean_loss_f<=best_f_loss:
        #     best_f_loss = mean_loss_f
        #     best_save_path = os.path.join("text_f", 'best')
        #     best_save_path = best_saver_faces.save(sess, best_save_path, global_step=e + 1)
        #     print("- Found new best face model, saving in {}".format(best_save_path))
               
        #SAmples
        if e>125 and e % check_step_samples==0:
            sess.run(iterator_init_op_val)
            # sess.run(f_iterator_init_op_val)
            v_samples_np, f_samples_np, b_np = sess.run((vertex_samples_val, face_samples_val, vertex_model_batch_val))
            mesh_list = []
            for n in range(BATCH_SIZE):
                mesh_list.append({
                    'vertices': v_samples_np['vertices'][n][:v_samples_np['num_vertices'][n]],
                    'faces': data_utils.unflatten_faces(
                        f_samples_np['faces'][n][:f_samples_np['num_face_indices'][n]])
                    })
            data_utils.plot_meshes(mesh_list, ax_lims=0.5)
                

2023-07-23 10:54:32.653175: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1669] Found device 0 with properties: 
name: NVIDIA GeForce RTX 3090 major: 8 minor: 6 memoryClockRate(GHz): 1.74
pciBusID: 0000:65:00.0
2023-07-23 10:54:32.653221: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2023-07-23 10:54:32.653243: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2023-07-23 10:54:32.653250: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2023-07-23 10:54:32.653256: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcurand.so.10
2023-07-23 10:54:32.653263: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcusolver.so.11
2023-07-23 10:54:32.653269: I tensorflow/stream_executor/plat

Epoch 98/500


  0%|                                                                          | 0/226 [00:00<?, ?it/s]2023-07-23 10:55:34.648916: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
/cluster/51/ala1s/deepmind-research/polygen/data_utils.py:162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(vertices), np.array(faces)
2023-07-23 10:55:35.773992: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
100%|█████████████████████████████████████████████████| 226/226 [04:32<00:00,  1.21s/it, loss=4947.168]


- Train loss vertex: 5170.384


100%|███████████████████████████████████████████████████| 42/42 [00:14<00:00,  2.97it/s, loss=3685.639]


...

100%|█████████████████████████████████████████████████| 226/226 [03:22<00:00,  1.11it/s, loss=6554.104]


- Train loss vertex: 4824.2886


100%|███████████████████████████████████████████████████| 42/42 [00:08<00:00,  4.95it/s, loss=3532.973]


- Eval loss vertex: 4220.7607
Epoch 142/500


 32%|████████████████▏                                 | 73/226 [01:02<01:58,  1.30it/s, loss=5739.374]